In [2]:
#Import libraries for doing image analysis
from skimage.io import imread
from skimage.transform import resize
from sklearn.ensemble import RandomForestClassifier as RF
import glob
import os
from sklearn import cross_validation
from sklearn.cross_validation import StratifiedKFold as KFold
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from matplotlib import colors
from pylab import cm
from skimage import segmentation
from skimage.morphology import watershed
from skimage import measure
from skimage import morphology
import numpy as np
import pandas as pd
from scipy import ndimage
from skimage.feature import peak_local_max
import tensorflow as tf
from math import ceil, floor
import time

# make graphics inline
%matplotlib inline

/home/davide/miniconda3/envs/honours/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import warnings
warnings.filterwarnings("ignore")

#### Importing the Data

In [4]:
# get the classnames from the directory structure
directory_names = list(set(glob.glob(os.path.join("competition_data","train", "*"))\
 ).difference(set(glob.glob(os.path.join("competition_data","train","*.*")))))

#### Preparing Training Data


In [5]:
# Rescale the images and create the combined metrics and training labels

#get the total training images
numberofImages = 0
for folder in directory_names:
    for fileNameDir in os.walk(folder):   
        for fileName in fileNameDir[2]:
             # Only read in the images
            if fileName[-4:] != ".jpg":
              continue
            numberofImages += 1
            
# We'll rescale the images to be 25x25
maxPixel = 28
imageSize = maxPixel * maxPixel
num_rows = numberofImages # one row for each image in the training dataset
num_features = imageSize # + 1 # for our ratio

# X is the feature vector with one row of features per image
# consisting of the pixel values and our metric
X = np.zeros((num_rows, num_features), dtype=float)
# y is the numeric class label 
y = np.zeros((num_rows))

files = []
# Generate training data
i = 0    
label = 0
# List of string of class names
namesClasses = list()

print("Reading images")
# Navigate through the list of directories
for folder in directory_names:
    # Append the string class name for each class
    currentClass = folder.split(os.pathsep)[-1]
    namesClasses.append(currentClass)
    for fileNameDir in os.walk(folder):   
        for fileName in fileNameDir[2]:
            # Only read in the images
            if fileName[-4:] != ".jpg":
              continue

            # Read in the images and create the features
            nameFileImage = "{0}{1}{2}".format(fileNameDir[0], os.sep, fileName)            
            image = imread(nameFileImage, as_grey=True)
            files.append(nameFileImage)
            #axisratio = getMinorMajorRatio(image)
            image = resize(image, (maxPixel, maxPixel))

            # Store the rescaled image pixels and the axis ratio
            X[i, 0:imageSize] = np.reshape(image, (1, imageSize))
            #X[i, imageSize] = axisratio

            # Store the classlabel
            y[i] = label
            i += 1
            # report progress for each 5% done  
            report = [int((j+1)*num_rows/10.) for j in range(10)]
            if i in report: 
                print (np.ceil(i *100.0 / num_rows), "% done")
    label += 1

Reading images
10.0 % done
20.0 % done
30.0 % done
40.0 % done
50.0 % done
60.0 % done
70.0 % done
80.0 % done
90.0 % done
100.0 % done


## Augmenting Dataset

In [6]:
IMAGE_SIZE = maxPixel

In [7]:
def flip_images(X_imgs):
    X_flip = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (IMAGE_SIZE, IMAGE_SIZE, 1))
    tf_img1 = tf.image.flip_left_right(X)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for img in X_imgs:
            flipped_imgs = sess.run([tf_img1], feed_dict = {X: img})
            X_flip.extend(flipped_imgs)
    X_flip = np.array(X_flip, dtype = np.float32)
    return np.concatenate((X_imgs, X_flip))

In [8]:
def rotate_images(X_imgs):
    X_rotate = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (IMAGE_SIZE, IMAGE_SIZE, 1))
    k = tf.placeholder(tf.int32)
    tf_img = tf.image.rot90(X, k = k)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for img in X_imgs:
            for i in range(3):  # Rotation at 90, 180 and 270 degrees
                rotated_img = sess.run(tf_img, feed_dict = {X: img, k: i + 1})
                X_rotate.append(rotated_img)
        
    X_rotate = np.array(X_rotate, dtype = np.float32)
    return np.concatenate((X_imgs, X_rotate))

In [9]:
def scale_images(X_imgs, scales=[0.90]):
    # Various settings needed for Tensorflow operation
    boxes = np.zeros((len(scales), 4), dtype = np.float32)
    for index, scale in enumerate(scales):
        x1 = y1 = 0.5 - 0.5 * scale # To scale centrally
        x2 = y2 = 0.5 + 0.5 * scale
        boxes[index] = np.array([y1, x1, y2, x2], dtype = np.float32)
    box_ind = np.zeros((len(scales)), dtype = np.int32)
    crop_size = np.array([maxPixel, maxPixel], dtype = np.int32)
    
    X_scale_data = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (1, maxPixel, maxPixel, 1))
    # Define Tensorflow operation for all scales but only one base image at a time
    tf_img = tf.image.crop_and_resize(X, boxes, box_ind, crop_size)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for img_data in X_imgs:
            batch_img = np.expand_dims(img_data, axis = 0)
            scaled_imgs = sess.run(tf_img, feed_dict = {X: batch_img})
            X_scale_data.extend(scaled_imgs)
    
    X_scale_data = np.array(X_scale_data, dtype = np.float32)
    return np.concatenate((X_imgs, X_scale_data))

In [10]:
def get_translate_parameters(index):
    if index == 0: # Translate left 20 percent
        offset = np.array([0.0, 0.2], dtype = np.float32)
        size = np.array([IMAGE_SIZE, ceil(0.8 * IMAGE_SIZE)], dtype = np.int32)
        w_start = 0
        w_end = int(ceil(0.8 * IMAGE_SIZE))
        h_start = 0
        h_end = IMAGE_SIZE
    elif index == 1: # Translate right 20 percent
        offset = np.array([0.0, -0.2], dtype = np.float32)
        size = np.array([IMAGE_SIZE, ceil(0.8 * IMAGE_SIZE)], dtype = np.int32)
        w_start = int(floor((1 - 0.8) * IMAGE_SIZE))
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = IMAGE_SIZE
    elif index == 2: # Translate top 20 percent
        offset = np.array([0.2, 0.0], dtype = np.float32)
        size = np.array([ceil(0.8 * IMAGE_SIZE), IMAGE_SIZE], dtype = np.int32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = int(ceil(0.8 * IMAGE_SIZE)) 
    else: # Translate bottom 20 percent
        offset = np.array([-0.2, 0.0], dtype = np.float32)
        size = np.array([ceil(0.8 * IMAGE_SIZE), IMAGE_SIZE], dtype = np.int32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = int(floor((1 - 0.8) * IMAGE_SIZE))
        h_end = IMAGE_SIZE 
        
    return offset, size, w_start, w_end, h_start, h_end

def translate_images(X_imgs):
    offsets = np.zeros((len(X_imgs), 2), dtype = np.float32)
    n_translations = 2
    X_translated_arr = []
    
    tf.reset_default_graph()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(n_translations):
            X_translated = np.zeros((len(X_imgs), IMAGE_SIZE, IMAGE_SIZE, 1), 
				    dtype = np.float32)
            X_translated.fill(1.0) # Filling background color
            base_offset, size, w_start, w_end, h_start, h_end = get_translate_parameters(i)
            offsets[:, :] = base_offset 
            glimpses = tf.image.extract_glimpse(X_imgs, size, offsets)
            
            glimpses = sess.run(glimpses)
            X_translated[:, h_start: h_start + size[0], \
			 w_start: w_start + size[1], :] = glimpses
            X_translated_arr.extend(X_translated)
    X_translated_arr = np.array(X_translated_arr, dtype = np.float32)
    return np.concatenate((X_imgs, X_translated_arr))

In [11]:
def augment_dataset(X_imgs, y_imgs):
    assert len(X_imgs) == len(y_imgs)
    X_aug = []
    y_aug = []
    i = 0
    print("Starting Dataset Augmentation...")
    
    for i in range(len(y_imgs)):
        
        imgs = [np.reshape(X_imgs[i,:],(IMAGE_SIZE,IMAGE_SIZE,1))]
        label = [y_imgs[i]]
        
        imgs = flip_images(imgs)
        imgs = rotate_images(imgs)
        imgs = scale_images(imgs)
        imgs = translate_images(imgs)
        
        labels = label * len(imgs)
#         print(len(labels), imgs.shape)
        
        X_aug += [np.array(x) for x in imgs.tolist()[:]]
        y_aug += labels
        
        i += 1
        # report progress for each 5% done  
        report = [int((j+1)*len(y_imgs)/10.) for j in range(10)]
        if i in report: 
            print (np.ceil(i *100.0 / len(y_imgs)), "% done")
        
    return X_aug, y_aug

## CNN for image classification

In [12]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [13]:
N_CLASSES = len(set(y))
print(N_CLASSES)

121


In [14]:
def cnn_model_fn(features, labels, mode):
    
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # Plankton images are 25x25 pixels, and have one color channel
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=128,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 7, 7, 64]
    # Output Tensor Shape: [batch_size, 7 * 7 * 64]
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 128])

    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    
    dense2 = tf.layers.dense(inputs=dropout, units=400, activation=tf.nn.relu)
    
    dropout2 = tf.layers.dropout(
      inputs=dense2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout2, units=121) # N_CLASSES

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [15]:
#Generate indexes to shuffle dataset and create testset

idx = np.arange(len(y))
np.random.seed(seed=1234)
np.random.shuffle(idx)
separator = ceil(len(y)/10*8)
print("Train / Total: ", separator, "/", len(y))

train_data = X[idx[:separator]].astype('float32')
eval_data = X[idx[separator:]].astype('float32')
train_labels = y[idx[:separator]].astype(int)
eval_labels = y[idx[separator:]].astype(int)

train_data = train_data #[:, 0:-1] # Returns np.array, remove axis rateo feature
eval_data = eval_data #[:, 0:-1]

print("Shapes:", train_data.shape, train_labels.shape, eval_data.shape, eval_labels.shape)

Train / Total:  24269 / 30336
Shapes: (24269, 784) (24269,) (6067, 784) (6067,)


In [25]:
def run_experiment(train_data, train_labels, eval_data, eval_labels, n_step=40000):
    # Create the Estimator
    now = str(time.time())[-5:]
    image_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/models/convnet_model_whole_22231") # convnet_model_whole_"+now)

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
    image_classifier.train(
      input_fn=train_input_fn,
      steps=n_step)#,
      #hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
    eval_results = image_classifier.evaluate(input_fn=eval_input_fn)
#     print(eval_results)
    return eval_results, image_classifier

In [20]:
start_t = time.time()
basic_results, image_classifier = run_experiment(train_data, train_labels, eval_data, eval_labels)
print("Finish time: ",str(time.time() - start_t))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/models/convnet_model_whole_40589', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9170748b70>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/models/convnet_model_whole_40589/model.ckpt.
INFO:tensorflow:loss = 4.79702, step = 1
INFO:tensorflow:global_step/sec: 25.5653
INFO:tensorflow:loss = 4.78872, step = 101 (3.914 sec)
INFO:tensorflow:global_step/sec: 25.7358
INFO:tensorflow:loss = 4.74213, step = 201 (3.885 sec)
INFO:tensorflow:global_step

INFO:tensorflow:global_step/sec: 25.8684
INFO:tensorflow:loss = 3.82256, step = 7801 (3.866 sec)
INFO:tensorflow:global_step/sec: 25.8352
INFO:tensorflow:loss = 3.88515, step = 7901 (3.871 sec)
INFO:tensorflow:global_step/sec: 25.8592
INFO:tensorflow:loss = 4.24745, step = 8001 (3.867 sec)
INFO:tensorflow:global_step/sec: 25.8645
INFO:tensorflow:loss = 3.78953, step = 8101 (3.866 sec)
INFO:tensorflow:global_step/sec: 25.8658
INFO:tensorflow:loss = 4.06327, step = 8201 (3.866 sec)
INFO:tensorflow:global_step/sec: 25.8764
INFO:tensorflow:loss = 3.95499, step = 8301 (3.865 sec)
INFO:tensorflow:global_step/sec: 25.8491
INFO:tensorflow:loss = 4.27475, step = 8401 (3.868 sec)
INFO:tensorflow:global_step/sec: 25.8625
INFO:tensorflow:loss = 3.92606, step = 8501 (3.867 sec)
INFO:tensorflow:global_step/sec: 25.8433
INFO:tensorflow:loss = 4.00665, step = 8601 (3.869 sec)
INFO:tensorflow:global_step/sec: 25.8596
INFO:tensorflow:loss = 4.11207, step = 8701 (3.867 sec)
INFO:tensorflow:global_step/se

INFO:tensorflow:global_step/sec: 25.8629
INFO:tensorflow:loss = 3.52341, step = 16101 (3.866 sec)
INFO:tensorflow:global_step/sec: 25.8366
INFO:tensorflow:loss = 3.47119, step = 16201 (3.870 sec)
INFO:tensorflow:global_step/sec: 25.8303
INFO:tensorflow:loss = 3.45064, step = 16301 (3.872 sec)
INFO:tensorflow:global_step/sec: 25.8656
INFO:tensorflow:loss = 3.6, step = 16401 (3.866 sec)
INFO:tensorflow:global_step/sec: 25.8442
INFO:tensorflow:loss = 3.50293, step = 16501 (3.870 sec)
INFO:tensorflow:global_step/sec: 25.8656
INFO:tensorflow:loss = 3.82524, step = 16601 (3.867 sec)
INFO:tensorflow:global_step/sec: 25.8435
INFO:tensorflow:loss = 3.43887, step = 16701 (3.869 sec)
INFO:tensorflow:global_step/sec: 25.8593
INFO:tensorflow:loss = 3.42947, step = 16801 (3.867 sec)
INFO:tensorflow:global_step/sec: 25.8473
INFO:tensorflow:loss = 3.6236, step = 16901 (3.869 sec)
INFO:tensorflow:global_step/sec: 25.8527
INFO:tensorflow:loss = 3.50331, step = 17001 (3.868 sec)
INFO:tensorflow:global_st

INFO:tensorflow:global_step/sec: 25.9993
INFO:tensorflow:loss = 3.32746, step = 24501 (3.846 sec)
INFO:tensorflow:global_step/sec: 25.9637
INFO:tensorflow:loss = 3.15417, step = 24601 (3.851 sec)
INFO:tensorflow:global_step/sec: 25.9916
INFO:tensorflow:loss = 2.97151, step = 24701 (3.848 sec)
INFO:tensorflow:global_step/sec: 25.9878
INFO:tensorflow:loss = 3.11006, step = 24801 (3.848 sec)
INFO:tensorflow:global_step/sec: 25.9782
INFO:tensorflow:loss = 3.35923, step = 24901 (3.849 sec)
INFO:tensorflow:global_step/sec: 25.979
INFO:tensorflow:loss = 2.89433, step = 25001 (3.849 sec)
INFO:tensorflow:global_step/sec: 25.9772
INFO:tensorflow:loss = 2.95261, step = 25101 (3.850 sec)
INFO:tensorflow:global_step/sec: 25.9693
INFO:tensorflow:loss = 2.74649, step = 25201 (3.851 sec)
INFO:tensorflow:global_step/sec: 25.9799
INFO:tensorflow:loss = 3.33674, step = 25301 (3.849 sec)
INFO:tensorflow:global_step/sec: 25.9734
INFO:tensorflow:loss = 3.14545, step = 25401 (3.851 sec)
INFO:tensorflow:globa

INFO:tensorflow:global_step/sec: 25.9963
INFO:tensorflow:loss = 2.8722, step = 32801 (3.847 sec)
INFO:tensorflow:global_step/sec: 25.9828
INFO:tensorflow:loss = 3.01168, step = 32901 (3.849 sec)
INFO:tensorflow:global_step/sec: 25.9695
INFO:tensorflow:loss = 2.61785, step = 33001 (3.851 sec)
INFO:tensorflow:global_step/sec: 25.9862
INFO:tensorflow:loss = 2.71534, step = 33101 (3.848 sec)
INFO:tensorflow:global_step/sec: 25.9989
INFO:tensorflow:loss = 3.15498, step = 33201 (3.846 sec)
INFO:tensorflow:global_step/sec: 25.9716
INFO:tensorflow:loss = 2.78956, step = 33301 (3.850 sec)
INFO:tensorflow:global_step/sec: 25.9829
INFO:tensorflow:loss = 2.81055, step = 33401 (3.849 sec)
INFO:tensorflow:global_step/sec: 25.9851
INFO:tensorflow:loss = 2.66898, step = 33501 (3.848 sec)
INFO:tensorflow:global_step/sec: 25.9646
INFO:tensorflow:loss = 2.76849, step = 33601 (3.851 sec)
INFO:tensorflow:global_step/sec: 25.9754
INFO:tensorflow:loss = 2.96886, step = 33701 (3.850 sec)
INFO:tensorflow:globa

In [21]:
start_t = time.time()

train_data_AUGM, train_labels_AUGM = augment_dataset(train_data, train_labels)
print("Augmented data", str(time.time() - start_t))
train_data_AUGM = np.array(train_data_AUGM, dtype='float32')
train_labels_AUGM = np.array(train_labels_AUGM, dtype='int')

print("Converted data", str(time.time() - start_t))

print("Augmented shape: ", train_data_AUGM.shape)

Starting Dataset Augmentation...
10.0 % done
20.0 % done
30.0 % done
40.0 % done
50.0 % done
60.0 % done
70.0 % done
80.0 % done
90.0 % done
100.0 % done
Augmented data 2990.426670074463
Converted data 6290.987701416016
Augmented shape:  (1164912, 28, 28, 1)


In [33]:
start_t = time.time()
augmented_results, image_classifier = run_experiment(train_data_AUGM, train_labels_AUGM, eval_data, eval_labels, n_step=600000) # 40000*48)
print("Finish time: ",str(time.time() - start_t))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/models/convnet_model_whole_22231', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8f0206b240>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/models/convnet_model_whole_22231/model.ckpt-659681
INFO:tensorflow:Saving checkpoints for 659682 into /tmp/models/convnet_model_whole_22231/model.ckpt.
INFO:tensorflow:loss = 1.36302, step = 659682
INFO:tensorflow:global_step/sec: 25.1698
INFO:tensorflow:loss = 1.34186, step = 659782 (3.978 sec)
INFO:tensorf

In [25]:
print("Training with 10000 steps")
print("BASIC RESULTS:  {'accuracy': 0.20224163, 'loss': 3.4980001, 'global_step': 10000}")
print("AUGMENTED RESULTS:  {'accuracy': 0.11719136, 'loss': 3.9900112, 'global_step': 10000}")

Training with 10000 steps
BASIC RESULTS:  {'accuracy': 0.20224163, 'loss': 3.4980001, 'global_step': 10000}
AUGMENTED RESULTS:  {'accuracy': 0.11719136, 'loss': 3.9900112, 'global_step': 10000}


In [1]:
print("Training with 50000 steps")
print("BASIC RESULTS:  {'accuracy': 0.45475525, 'loss': 2.0939481, 'global_step': 50000}")
print("AUGMENTED RESULTS:  {'accuracy': 0.23718476, 'loss': 3.0984316, 'global_step': 50000}")

Training with 50000 steps
BASIC RESULTS:  {'accuracy': 0.45475525, 'loss': 2.0939481, 'global_step': 50000}
AUGMENTED RESULTS:  {'accuracy': 0.23718476, 'loss': 3.0984316, 'global_step': 50000}


In [34]:
print("Final Training with:")
print("BASIC RESULTS with 160 epochs: ", basic_results)
print("AUGMENTED RESULTS with 56 epochs: ", augmented_results)

Final Training with:
BASIC RESULTS with 160 epochs:  {'accuracy': 0.37844074, 'loss': 2.4359987, 'global_step': 40000}
AUGMENTED RESULTS with 56 epochs:  {'accuracy': 0.54920059, 'loss': 1.5524381, 'global_step': 660010}
